In [1]:
import pandas as pd
from pydantic import BaseModel
from datetime import datetime, timedelta

In [2]:
invoices = { 
    "ACME INC.": [
        {
            "id": "12343",
            "amount_due": 10.0,
            "date_due": datetime.utcnow() + timedelta(days=10)
        },
        {
            "id": "12743",
            "amount_due": 20.0,
            "date_due": datetime.utcnow() + timedelta(days=78)
        },
        {
            "id": "12623",
            "amount_due": 30.0,
            "date_due": datetime.utcnow() + timedelta(days=2)
        },
        {
            "id": "127563",
            "amount_due": 30.0,
            "date_due": datetime.utcnow() + timedelta(days=99)
        },
    ],
    "LA Lakers": [
        {
            "id": "454336",
            "amount_due": 100.0,
            "date_due": datetime.utcnow() + timedelta(days=10)
        },
        {
            "id": "534654",
            "amount_due": 1020.0,
            "date_due": datetime.utcnow() + timedelta(days=78)
        },
        {
            "id": "25435",
            "amount_due": 1243.0,
            "date_due": datetime.utcnow() + timedelta(days=2)
        },
        {
            "id": "375546",
            "amount_due": 890.0,
            "date_due": datetime.utcnow() + timedelta(days=99)
        },
    ]
}

In [3]:
aging_cols = ["0-30", "31-60", "61-90", "90+"]
cols = ["Number", "Due Date"] + aging_cols

In [4]:
def datetime_to_aging_col(dt: datetime) -> str:
    time_from_now = dt - datetime.utcnow()
    
    if time_from_now <= timedelta(days=30):
        return "0-30"
    elif time_from_now <= timedelta(days=60):
        return "31-60"
    elif time_from_now <= timedelta(days=90):
        return "61-90"
    else:
        return "90+"

In [5]:
def invoice_to_row_dict(invoice):
    row_dict = {c: "0" for c in aging_cols}
    
    row_dict["Number"] = invoice['id']
    row_dict["Due Date"] = invoice['date_due'].strftime("%m/%d/%Y")
    
    bucket = datetime_to_aging_col(invoice['date_due'])
    row_dict[bucket] = f"${invoice['amount_due']}"
    
    return row_dict

In [6]:
def gen_total_row_dict(invoices, title: str = None):
    values = {c:[] for c in aging_cols}
    for i in invoices:
        col = datetime_to_aging_col(i['date_due'])
        values[col].append(i['amount_due'])
    
    row_dict = {c: f"${sum(values[c])}" for c in values}
    row_dict['Due Date'] = title if title else ''
    
    return row_dict
    

In [7]:
def gen_empty_row_df(df: pd.DataFrame) -> pd.DataFrame:
    return pd.DataFrame([[''] * len(df.columns)], columns=df.columns)

In [26]:
df = pd.DataFrame(columns=cols)

# Add Vendor Sections
for vendor_name, invoice_list in invoices.items():
    df = df.append(pd.DataFrame([[vendor_name] + [' ' for _ in range(len(cols)-1)]], columns=cols))
    
    invoice_rows = pd.DataFrame([invoice_to_row_dict(i) for i in invoice_list])
    sub_total_row = pd.DataFrame([gen_total_row_dict(invoice_list, title="subtotals:")])
    
    df = df.append(invoice_rows, sort=False)
    df = df.append(sub_total_row, sort=False)
    df = df.append(gen_empty_row_df(df), sort=False)
    

# Add Final Total
all_invoices = []
for invoice_list in invoices.values():
    all_invoices.extend(invoice_list)
    
total_row = pd.DataFrame([gen_total_row_dict(all_invoices, title="Totals:")])
df = df.append(total_row, sort=False)

# Ensure we don't see any NaNs
df.fillna('', inplace=True)
df

,Number,Due Date,0-30,31-60,61-90,90+
0,ACME INC.,,,,,
0,12343,04/13/2022,$10.0,0,0,0
1,12743,06/20/2022,0,0,$20.0,0
2,12623,04/05/2022,$30.0,0,0,0
3,127563,07/11/2022,0,0,0,$30.0
0,,subtotals:,$40.0,$0,$20.0,$30.0
0,,,,,,
0,LA Lakers,,,,,
0,454336,04/13/2022,$100.0,0,0,0
1,534654,06/20/2022,0,0,$1020.0,0


In [32]:
# Add preheader lines
with open('test3.csv', 'w') as fp:
    fp.write('April 3 2022 - Accounts Payable Aging Report,\n,\n')
    df.to_csv(fp, index=False)